In [13]:
import numpy as np
import tensorflow as tf
filePath = './test_case_ans/sample_case002_input.txt'
resPath = './test_case_ans/sample_case002_ans.txt'
c = 3e8

def loadData(filePath):
    with open(filePath, 'r') as fd:
        station_num = int(fd.readline())
        device_num = int(fd.readline())
        dimen = int(fd.readline())
        station_cord = []
        for i in range(station_num):
            cord = fd.readline().split()
            cord = [np.float64(i) for i in cord]
            station_cord.append(cord)
        station_cord = np.array(station_cord)

        device_cord = []
        for i in range(device_num):
            cord = fd.readline().split()
            cord = [np.float64(i) for i in cord]
            device_cord.append(cord)
        device_cord = np.array(device_cord)
        return station_cord, device_cord

def loadRes(filePath):
    with open(filePath, 'r') as fd:
        device_cord = []
        for line in fd:
            cord = [np.float64(i) for i in line.split()]
            device_cord.append(cord)
    device_cord = np.array(device_cord)
    return device_cord

    

S, D = loadData(filePath)
device_Dist = D*c

cord_init = np.mean(S, axis=0)
cord_init = cord_init.reshape([1, 3]) * np.ones([len(D), 3])
bias_init = np.zeros(len(D))
# cord_init = loadRes(resPath)

D = D[:10]
device_Dist = device_Dist[:10]
cord_init = cord_init[:10]
bias_init = bias_init[:10]
with tf.device('/gpu:2'):
#     cord_x = tf.Variable(cord_init[:, 0], name='Cordinate_x', trainable=False) #(N,)
#     cord_y = tf.Variable(cord_init[:, 1], name='Cordinate_y', trainable=False)
#     cord_z = tf.Variable(cord_init[:, 2], name='Cordinate_z', trainable=False)
    cord_x = tf.Variable(cord_init[:, 0], name='Cordinate_x') #(N,)
    cord_y = tf.Variable(cord_init[:, 1], name='Cordinate_y')
    cord_z = tf.Variable(cord_init[:, 2], name='Cordinate_z')

    Station_x = tf.constant(S[:, 0], tf.float64) #(M,)
    Station_y = tf.constant(S[:, 1], tf.float64)
    Station_z = tf.constant(S[:, 2], tf.float64)

    bias = tf.Variable(bias_init, 'Bias') #(N, )

    alpha1 = tf.Variable(np.ones(len(cord_init)), 'Alpha1', dtype = tf.float64)  #(N,)
    alpha2 = tf.Variable(np.ones(len(cord_init)), 'Alpha2', dtype = tf.float64)  #(N,)
    alpha3 = tf.Variable(np.ones(len(cord_init)), 'Alpha3', dtype = tf.float64)  #(N,)
#     spd = tf.Variable(3e8, 'SpeedOfLight', dtype = tf.float64)  #(N,)
    D_dist = tf.constant(device_Dist, tf.float64) 

expand_cord_x = tf.expand_dims(cord_x, 1) #(N, 1)
expand_cord_y = tf.expand_dims(cord_y, 1)
expand_cord_z = tf.expand_dims(cord_z, 1)

expand_Station_x = tf.expand_dims(Station_x, 0) #(1, M)
expand_Station_y = tf.expand_dims(Station_y, 0)
expand_Station_z = tf.expand_dims(Station_z, 0)

expand_alpha1 = tf.expand_dims(alpha1, 1) #(N, 1)

expand_alpha2 = tf.expand_dims(alpha2, 1) 
expand_alpha3 = tf.expand_dims(alpha3, 1)

expand_bias = tf.expand_dims(bias, 1)

clip_cord_z = tf.minimum(tf.maximum(expand_cord_z, 0), expand_Station_z)

dist_x = (expand_cord_x - expand_Station_x) **2 #(N, M)
dist_y = (expand_cord_y - expand_Station_y) **2
dist_z = (expand_cord_z - expand_Station_z) **2

global_step = tf.Variable(0, name='global_step', trainable=False)


# dist_hat = tf.sqrt(expand_alpha1**2 * dist_x +expand_alpha2**2 * dist_y +
#                    expand_alpha3**2 * dist_z) - expand_bias #(N,M)
# dist_hat = expand_alpha1 * tf.sqrt(dist_x + dist_y + expand_alpha2**2) - expand_bias #(N,M)
dist_hat = (expand_alpha1+expand_alpha1**2+expand_alpha1**3) * tf.sqrt(dist_x + dist_y + dist_z) - expand_bias  #(N,M)

Z_g = expand_Station_z - tf.sqrt(tf.maximum(((D_dist + expand_bias)/expand_alpha1) **2 -dist_x-dist_y, 0))

losses = tf.reduce_sum((dist_hat - D_dist) **2, 1)
learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.85, staircase=True) + tf.train.exponential_decay(1e-2, global_step, 5000, 0.85, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op1 = optimizer.minimize(losses, global_step=global_step , var_list=[cord_x, cord_y, alpha1, alpha2,alpha3])

optimizer2 = tf.train.AdamOptimizer(learning_rate=learning_rate*10)
train_op2 = optimizer2.minimize(losses, global_step=global_step, var_list=[cord_z])

train_op = tf.group(train_op1, train_op2)

config=tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
with tf.Session(config=config) as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(30000):
        _, loss = sess.run([train_op, losses])
        if i%1000 == 0:
            print np.sum(loss), 'lr=', sess.run(learning_rate)
    B, Px, Py, Pzz, a1, a2, a3= sess.run([bias, cord_x, cord_y, cord_z, alpha1, alpha2, alpha3])
    light_speed = sess.run(spd)
    Pz = sess.run(Z_g)

126879648.587 lr= 0.51
214655.544242 lr= 0.37125
354079.760687 lr= 0.271003
2355.70233209 lr= 0.197075
57060.4066903 lr= 0.144745
5763.97000165 lr= 0.105662
13530.7426452 lr= 0.0783459
34.4951714586 lr= 0.0586099
34.8374572266 lr= 0.0432668
130.77056105 lr= 0.0329645
739.897886336 lr= 0.0245998
215.983265036 lr= 0.019222
37.2828617892 lr= 0.0153364
57.0505205298 lr= 0.0117461
66.0019881059 lr= 0.00971786
34.4256452356 lr= 0.00758688
113.703065888 lr= 0.00652811
80.0831742756 lr= 0.00576315
36.0652057389 lr= 0.00464474
34.637869596 lr= 0.00424543
70.8994847237 lr= 0.00347606
39.7010724995 lr= 0.00326761
34.9704934244 lr= 0.00311701
52.1043857871 lr= 0.00259947
37.2711208181 lr= 0.00252085
48.7244774534 lr= 0.00211663
34.6979951223 lr= 0.00207559
39.798008207 lr= 0.00204594
35.6285676421 lr= 0.00172921
41.2291954837 lr= 0.00171373


In [4]:
print light_speed

3.59510681488e-07


In [14]:
with open('ans_2.txt', 'w') as fd:
    for i in range(len(Px)):
        fd.write('{}\t{}\t{}\n'.format(Px[i], Py[i], Pz[i]))
for i in range(20):
    print '{:3}\t{:3}\t{:3}'.format(Px[i], Py[i], Pzz[i])
print '-------'
print B[:20]
print '-------'
print a1[:20]

-259.452615063	-341.958914278	-7.58372496016
305.036767352	-277.163861244	-4.94149883643
332.896605381	-223.355112356	-5.18848474724
165.879293006	-235.614892414	-4.63732674744
-293.418666559	230.957254975	-3.03277334758
-103.483808208	11.1800382612	-3.79539151235
-32.5728365187	-261.130739425	-7.81910049365
-249.376289349	153.489154717	-5.34818661508
-200.624137877	279.277219551	-4.28223634471
198.130838024	263.398520636	7.42715019569


IndexError: index 10 is out of bounds for axis 0 with size 10

In [2]:
%%bash
head -10 test_case_ans/sample_case002_ans.txt
head -22 test_case_ans/sample_case002_input.txt

-259.20	-341.86	1.12	
304.83	-276.83	1.48	
332.86	-223.33	1.71	
165.98	-235.58	1.30	
-293.23	230.90	1.24	
-103.55	11.09	1.15	
-32.56	-260.88	1.40	
-249.33	153.47	1.86	
-200.53	279.14	1.76	
198.23	263.55	1.01	
40
1200
3
299.46	335.64	3.77	
201.41	167.37	4.70	
-303.59	-333.12	2.36	
36.52	118.45	3.82	
-294.49	208.39	2.77	
367.06	-294.69	3.55	
249.87	-144.93	3.81	
379.16	128.55	3.40	
-11.85	65.16	4.07	
-278.27	42.56	3.33	
189.71	264.29	2.99	
-153.28	-222.34	2.50	
117.18	-221.11	2.27	
-248.07	-74.31	2.50	
70.78	141.55	2.04	
-348.28	146.78	3.80	
157.31	318.52	4.71	
295.36	-175.00	2.14	
-229.84	205.91	4.62	


In [3]:
import numpy as np
aa = np.array([[1, 2, 3], [4, 5, 6]])
print aa[:, 1].shape

(2,)
